# 1. Linear SVM 분류

### 1. Soft margin Classifier

Scikit learn API

- C값이 줄이면 마진의 폭이 넓어지지만 마진 오류도 커진다.

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2,3)] # 꽃잎 길이, 꽃잎 너비
y = (iris['target'] == 2).astype(np.float64)

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss='hinge')),
])

svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [2]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

- SVM 분류기는 로지스틱 회귀와 다르게 클래스에 대한 확률을 제공하지 않는다.

- 다른 방법으로는 SVC(kernel='linear', C=1)과 같이 SVC 모델을 사용할 수 있다. 하지만 큰 훈련 세트에서는 속도가 느리므로 권장하지 않는다.
- 또 다른 방법으로는 SGDClassifier(loss='hinge', alpha=1/(m * C))와 같이 SGDClassifier 모델을 사용한다. 이는 선형 SVM 분류기를 훈련시키기 위해 일반적인 확률적 경사하강법을 적용합니다. LinearSVC 만큼빠르게 수렴하지 않지만 데이터셋이 아주 커서 메모리에 적재할 수 없거나, 온라인 학습으로 분류 문제를 다룰 떄는 용이합니다.

# 2. 비선형 SVM 분류

In [3]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('poly_features', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

## 2.1. 다항식 커널

In [4]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

## 2.2 유사도 측정

- guassian RBF(Radial Basis Function)

$$ \phi({x, l}) = \exp{(- \gamma ||x - l||^2 )}$$


- 데이터의 모든 샘플 위치에 랜드마크를 정하고 선형으로 분류한다.
- 단점은 m 개의 특성을 가진 m 개의 샘플이 m 개의 특성을 가진 m 개의 샘플로 변환된다는 것이다.

In [7]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.0001))
])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

- gamma가 커지면 결정 경계가 더 많이 휠 수 있다. (overfitting)
- gamma가 작아지면 결정 경계가 부드러워진다.

###### Tip: 언제나 선형 커널을 먼저 시도해봐야 합니다. LinearSVC가 SVC(kernel="linear") 보다 훨씬 빠르다. 특히 훈련 세트가 아주 크거나 특성 수가 많을 경우에 그렇다. 훈련 세트가 너무 크지 않다면 RBF 커널을 시도해보면 좋다. 대부분의 경우 이 커널이 잘 들어맞는다.

# 5.3 SVM 회귀

- 회귀에 적용하는 방법은 목표로 반대로 하는 것이다. SVM 분류에서는 일정한 마진 오류 안에서 두 클래스 간의 도로 폭이 가능한 최도록 하는 대신, SVM 회귀는 제한된 마진 오류 안에서 가능한 많은 샘플이 들어가도록 학습한다. 마진의 폭은 epsilon으로 조절한다. epsilon이 크면 마진이 크다.

- 마진 안에서는 훈련 샘플이 추가되어도 모델의 예측에는 영향이 없다. 그래서 이 모델은 e에 민감하지 않다고 한다.

In [8]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

### 비선형ㅎ ㅚ귀 작업

In [9]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='auto',
  kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)